In [ ]:
import sympy
sympy.init_printing(use_unicode=True)

# Lame parameters + density
la = sympy.symbols('lambda', real=True)
mu = sympy.symbols('mu', real=True)
rh = sympy.symbols('rho', real=True)
o1 = sympy.symbols('omega_1', real=True)
o2 = sympy.symbols('omega_2', real=True)

# left an right Lame parameters + density
lal = sympy.symbols('lambda_l', real=True)
lar = sympy.symbols('lambda_r', real=True)
mul = sympy.symbols('mu_l', real=True)
mur = sympy.symbols('mu_r', real=True)
rhl = sympy.symbols('rho_l', real=True)
rhr = sympy.symbols('rho_r', real=True)
o1l = sympy.symbols('o1_l', real=True)
o1r = sympy.symbols('o1_r', real=True)
o2l = sympy.symbols('o2_l', real=True)
o2r = sympy.symbols('o2_r', real=True)

la, mu, rh, o1, o2, lal, lar, mul, mur, rhl, rhr, o1l, o1r, o2l, o2r

In [ ]:
# elastic Jacobian matrix
Ae = sympy.Matrix(
 [
   [ 0, 0, 0,  0, 0, 0,  -(la+2*mu), 0, 0],
   [ 0, 0, 0,  0, 0, 0,  -la, 0, 0],
   [ 0, 0, 0,  0, 0, 0,  -la, 0, 0],
  
   [ 0, 0, 0,  0, 0, 0,  0, -mu, 0],
   [ 0, 0, 0,  0, 0, 0,  0, 0, 0],
   [ 0, 0, 0,  0, 0, 0,  0, 0, -mu],

   [ -1/rh, 0, 0,  0, 0, 0,  0, 0, 0],
   [ 0, 0, 0,  -1/rh, 0, 0,  0, 0, 0],
   [ 0, 0, 0,  0, 0, -1/rh,  0, 0, 0]
 ]
)

# viscoelastic Jacobian matrix
Av = sympy.Matrix(
 [
   [ 0, 0, 0,  0, 0, 0,  -1, 0, 0 ],
   [ 0, 0, 0,  0, 0, 0,  0, 0, 0 ],
   [ 0, 0, 0,  0, 0, 0,  0, 0, 0 ],
  
   [ 0, 0, 0,  0, 0, 0,  0, -sympy.Rational(1, 2), 0 ],
   [ 0, 0, 0,  0, 0, 0,  0, 0, 0,],
   [ 0, 0, 0,  0, 0, 0,  0, 0, -sympy.Rational(1, 2)]
 ]
)
Ae, Av

In [ ]:
Ae.eigenvects()

In [ ]:
#sympy.BlockMatrix( sympy.BlockMatrix( [ Ae, Av ] ), sympy.ZeroMatrix(15,1) )
nzBlock = sympy.Matrix( [Ae, o1*Av, o2*Av] )
jac = sympy.Matrix( [ nzBlock.transpose(), sympy.zeros(12,21) ] ).transpose()

sympy.factor( jac.eigenvects() )

In [ ]:
P, D = jac.diagonalize()

# replace by element-local value to account for material interfaces
for mat in [P, D]:
  for le in [3,4,7]:
    mat[:,12+le] = mat[:,12+le].subs( la, lal )
    mat[:,12+le] = mat[:,12+le].subs( mu, mul )
    mat[:,12+le] = mat[:,12+le].subs( rh, rhl )
    mat[:,12+le] = mat[:,12+le].subs( o1, o1l )
    mat[:,12+le] = mat[:,12+le].subs( o2, o2l )

  for le in [5,6,8]:
    mat[:,12+le] = mat[:,12+le].subs( la, lar )
    mat[:,12+le] = mat[:,12+le].subs( mu, mur )
    mat[:,12+le] = mat[:,12+le].subs( rh, rhr )
    mat[:,12+le] = mat[:,12+le].subs( o1, o1r )
    mat[:,12+le] = mat[:,12+le].subs( o2, o2r )
    
D, P

In [ ]:
import copy
Pn = copy.deepcopy(P)
Pn[:,0:15] = sympy.zeros(21,15)
Pn[:,17:19] = sympy.zeros(21,2)
Pn[:,20] = sympy.zeros(21,1)
Pn * P.inv()

In [ ]:
jacL = copy.deepcopy(jac)
jacL = jacL.subs( la, lal )
jacL = jacL.subs( mu, mul )
jacL = jacL.subs( rh, rhl )
jacL = jacL.subs( o1, o1l )
jacL = jacL.subs( o2, o2l )


#sympy.factor( jacL * Pn * P.inv() )[0:9,:], sympy.factor( jacL * Pn * P.inv() )[9:15,:], sympy.factor( jacL * Pn * P.inv() )[15:21,:]
sympy.factor( jacL * Pn * P.inv() )[9:15,0:9]

In [ ]:
#sympy.factor( jacL - jacL * Pn * P.inv() )[0:9,:], sympy.factor( jacL * Pn * P.inv() )[9:15,:], sympy.factor( jacL * Pn * P.inv() )[15:21,:]
sympy.factor( jacL - jacL * Pn * P.inv() )[9:15,0:9]

In [ ]:
# unit tests
sympy.N( ( jacL - jacL * Pn * P.inv() )[9:15,0:9].subs( lal, 20.8E9 ).subs( lar, 19.4E9  )\
                                                 .subs( mul, 10.4E9 ).subs( mur, 13.1E9) )\
                                                 .subs( rhl,  1.2E3 ).subs( rhr, 1.1E3   )\
                                                 .subs( o1l, 1 )

In [ ]:
sympy.N( (        jacL * Pn * P.inv() )[9:15,0:9].subs( lal, 20.8E9 ).subs( lar, 19.4E9  )\
                                                 .subs( mul, 10.4E9 ).subs( mur, 13.1E9) )\
                                                 .subs( rhl,  1.2E3 ).subs( rhr, 1.1E3   )\
                                                 .subs( o1l, 1 )